<a href="https://colab.research.google.com/github/Muhammad-javed440/Q-3/blob/master/tool_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install pakages
!pip install --upgrade -q langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.4 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
# GEMINI API KEY
GOOGLE_API_KEY= userdata.get("GEMINI_API_KEY")
# WETAHTER API KEY
WEATHER_API_KEY= userdata.get("WATHER_API_KEY")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [4]:
# configure model and
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key=GOOGLE_API_KEY # GEMINI API KEY
)

In [5]:
llm.invoke("Hi!")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-294f5909-c02a-4b80-90b9-dad035c8d965-0', usage_metadata={'input_tokens': 3, 'output_tokens': 11, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}})

In [6]:
from langchain_core.tools import tool
import requests

# decorator
@tool
def annual_return(a:int):
  """Returns annual return of a number"""
  return (a +15)/5

# decorator
@tool
def add_two_numbers(a:int, b :int):
  """Returns additional return of a number""" # this doc string
  return (a +b)


# decorator
@tool
def multi(a:int, b:int):
    """Returns multiply return of a number"""
    return( a*b)






@tool
def get_weather_data(city: str):
    """
    Fetches weather data for the given city using OpenWeatherMap API.

    Args:
        city (str): The name of the city to get the weather for.
        api_key (str): Your OpenWeatherMap API key.

    Returns:
        dict: A dictionary containing weather data for the city.
    """
    base_url = f"https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": WEATHER_API_KEY,
        "units": "metric"  # Use "imperial" for Fahrenheit
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        weather_data = response.json()
        return weather_data
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}



In [7]:
tools = [ annual_return, multi , add_two_numbers,get_weather_data]

In [8]:
response = llm.invoke("Whay is the annual return of 10?")
display(response.content)

'The phrase "annual return of 10" is ambiguous. It could mean a few different things, depending on the context. Here are the most common interpretations and how to understand them:\n\n**1. 10% Annual Return:**\n\n*   **Meaning:** This is the most likely interpretation. It means that an investment grows by 10% of its value each year.\n*   **Example:** If you invest $100, and have an annual return of 10%, you would have $110 at the end of the first year. In the second year, your $110 would grow by 10% to $121. This is compounding.\n*   **Calculation:** To calculate the value after a certain number of years, you can use the formula:\n    *   Future Value = Principal * (1 + Return Rate)^Number of Years\n    *   For example, after 5 years, the future value would be $100 * (1 + 0.10)^5 = $161.05 (approximately)\n\n**2. Return of 10 Units (Not Percentage):**\n\n*   **Meaning:** This is less common but could refer to a fixed return of 10 units (e.g., dollars, points, etc.) per year, regardless

In [10]:

llm_with_tools = llm.bind_tools(tools)

In [12]:
response = llm_with_tools.invoke("What is the anual return  of 20?")

response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'annual_return', 'arguments': '{"a": 20.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-549c1a4e-3929-4a5f-bfd1-f764c0a19b70-0', tool_calls=[{'name': 'annual_return', 'args': {'a': 20.0}, 'id': '29fec0a4-1c23-40a3-af27-43a4ebde1b6a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 247, 'output_tokens': 4, 'total_tokens': 251, 'input_token_details': {'cache_read': 0}})

In [18]:
from langchain_core.messages import HumanMessage
query = "Q1 What is the weather of murree ?, Q2 add two number 5 and 6?"

message = [HumanMessage(query)]

display(message)

[HumanMessage(content='Q1 What is the weather of murree ?, Q2 add two number 5 and 6?', additional_kwargs={}, response_metadata={})]

In [19]:
ai_msg = llm_with_tools.invoke(message)
message.append(tools)
message

[HumanMessage(content='Q1 What is the weather of murree ?, Q2 add two number 5 and 6?', additional_kwargs={}, response_metadata={}),
 [StructuredTool(name='annual_return', description='Returns annual return of a number', args_schema=<class 'langchain_core.utils.pydantic.annual_return'>, func=<function annual_return at 0x7d3a5c5c1750>),
  StructuredTool(name='multi', description='Returns multiply return of a number', args_schema=<class 'langchain_core.utils.pydantic.multi'>, func=<function multi at 0x7d3a5c5c1a20>),
  StructuredTool(name='add_two_numbers', description='Returns additional return of a number', args_schema=<class 'langchain_core.utils.pydantic.add_two_numbers'>, func=<function add_two_numbers at 0x7d3a89b6edd0>),
  StructuredTool(name='get_weather_data', description='Fetches weather data for the given city using OpenWeatherMap API.\n\nArgs:\n    city (str): The name of the city to get the weather for.\n    api_key (str): Your OpenWeatherMap API key.\n\nReturns:\n    dict: 

In [20]:
ai_msg.tool_calls

[{'name': 'get_weather_data',
  'args': {'city': 'murree'},
  'id': '0412f7d9-543a-489e-a05a-0fc71b34c1c4',
  'type': 'tool_call'},
 {'name': 'add_two_numbers',
  'args': {'a': 5.0, 'b': 6.0},
  'id': 'dae773aa-ddd9-487f-b5be-d7f3cb55cd0c',
  'type': 'tool_call'}]

In [21]:
for tool_call in      ai_msg.tool_calls:
  selected_tool={
      "add_two_numbers":add_two_numbers,
      "multiply":multi,
      "annual_return":annual_return,
      "get_weather_data":get_weather_data
  }[tool_call["name"].lower()]

  tool_msg = selected_tool.invoke(tool_call)
  # display(tool_msg)
  message.append(tool_msg)
display(message)

[HumanMessage(content='Q1 What is the weather of murree ?, Q2 add two number 5 and 6?', additional_kwargs={}, response_metadata={}),
 [StructuredTool(name='annual_return', description='Returns annual return of a number', args_schema=<class 'langchain_core.utils.pydantic.annual_return'>, func=<function annual_return at 0x7d3a5c5c1750>),
  StructuredTool(name='multi', description='Returns multiply return of a number', args_schema=<class 'langchain_core.utils.pydantic.multi'>, func=<function multi at 0x7d3a5c5c1a20>),
  StructuredTool(name='add_two_numbers', description='Returns additional return of a number', args_schema=<class 'langchain_core.utils.pydantic.add_two_numbers'>, func=<function add_two_numbers at 0x7d3a89b6edd0>),
  StructuredTool(name='get_weather_data', description='Fetches weather data for the given city using OpenWeatherMap API.\n\nArgs:\n    city (str): The name of the city to get the weather for.\n    api_key (str): Your OpenWeatherMap API key.\n\nReturns:\n    dict: 

In [23]:
ai_msg.tool_calls

[{'name': 'get_weather_data',
  'args': {'city': 'murree'},
  'id': '0412f7d9-543a-489e-a05a-0fc71b34c1c4',
  'type': 'tool_call'},
 {'name': 'add_two_numbers',
  'args': {'a': 5.0, 'b': 6.0},
  'id': 'dae773aa-ddd9-487f-b5be-d7f3cb55cd0c',
  'type': 'tool_call'}]